<a href="https://colab.research.google.com/github/dharmikmehta98/IMDB_Review_Classification_Using_BERT/blob/main/NLP_with_Bert_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP with Bert for Sentiment Analysis

### Importing the libraries
https://github.com/amaiya/ktrain#news-and-announcements

In [ ]:
!pip3 install ktrain

In [ ]:
import os.path
import numpy as np
import tensorflow as tf
import ktrain
from ktrain import text

## Part 1: Data Preprocessing

### Loading the IMDB dataset

In [ ]:
dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz",
                                  origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
                                  extract=True)
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), 'aclImdb')

In [ ]:
print(os.path.dirname(dataset))
print(IMDB_DATADIR)

### Creating the training and test sets

In [ ]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder(datadir=IMDB_DATADIR,
                                                                       classes=['pos','neg'],
                                                                       maxlen=500,
                                                                       train_test_names=['train','test'],
                                                                       preprocess_mode='bert')

## Part 2: Building the BERT model

In [ ]:
model = text.text_classifier(name='bert',
                             train_data=(x_train, y_train),
                             preproc=preproc)

## Part 3: Training the BERT model

In [ ]:
learner = ktrain.get_learner(model=model,
                             train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=6)

In [ ]:
learner.fit_onecycle(lr=2e-5,
                     epochs=1) 

# Part 4 : Save Trained Model

In [ ]:
#Ref : https://medium.com/analytics-vidhya/how-to-deploy-your-neural-network-model-using-ktrain-ae255b134c77

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('imdb_sentiment_model')
print('MODEL SAVED')

# Part 5:Load Trained Model

In [ ]:
predictor = ktrain.load_predictor('imdb_sentiment_model')

# Part 6: Predict the result

In [ ]:
predictor.predict(["not bad"])